In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
qb_url = "https://www.fantasypros.com/nfl/stats/qb.php?scoring=HALF"
rb_url = "https://www.fantasypros.com/nfl/stats/rb.php?scoring=HALF"
wr_url = "https://www.fantasypros.com/nfl/stats/wr.php?scoring=HALF"
te_url = "https://www.fantasypros.com/nfl/stats/te.php?scoring=HALF"
dst_url = "https://www.fantasypros.com/nfl/stats/dst.php?scoring=HALF"

adp_url = "https://www.fantasypros.com/nfl/adp/half-point-ppr-overall.php"

In [3]:
qb_proj_url = "https://www.fantasypros.com/nfl/projections/qb.php"
rb_proj_url = "https://www.fantasypros.com/nfl/projections/rb.php?week=draft"
wr_proj_url = "https://www.fantasypros.com/nfl/projections/wr.php?week=draft"
te_proj_url = "https://www.fantasypros.com/nfl/projections/te.php?week=draft"

qb_proj_df = pd.read_html(qb_proj_url, header=1)[0]
rb_proj_df = pd.read_html(rb_proj_url, header=1)[0]
wr_proj_df = pd.read_html(wr_proj_url, header=1)[0]
te_proj_df = pd.read_html(te_proj_url, header=1)[0]

all_proj_df = pd.concat([qb_proj_df[['Player','FPTS']], rb_proj_df[['Player','FPTS']], wr_proj_df[['Player','FPTS']], te_proj_df[['Player','FPTS']]])

In [4]:
te_proj_df.head()

,Player,REC,YDS,TDS,FL,FPTS
0,Travis Kelce KC,99.4,1179.8,9.3,0.5,172.7
1,Mark Andrews BAL,95.5,1187.6,8.9,0.5,171.3
2,Kyle Pitts ATL,73.1,997.9,4.7,0.5,126.7
3,Darren Waller LV,77.5,931.4,5.1,0.5,122.6
4,George Kittle SF,71.2,871.1,5.4,0.5,118.6


In [131]:
rb_proj_df['Proj FPTS'] = rb_proj_df['YDS']/10 + rb_proj_df['TDS']*6 + rb_proj_df['YDS.1']/10 + rb_proj_df['TDS.1']*6 - rb_proj_df['FL']*2 + rb_proj_df['REC']
qb_proj_df['Proj FPTS'] = qb_proj_df['YDS']/25 + qb_proj_df['TDS']*4 - qb_proj_df['INTS']*2 + qb_proj_df['YDS.1']/10 + qb_proj_df['TDS.1']*6 - qb_proj_df['FL']*2
wr_proj_df['Proj FPTS'] = wr_proj_df['YDS']/10 + wr_proj_df['TDS']*6 + wr_proj_df['YDS.1']/10 + wr_proj_df['TDS.1']*6 - wr_proj_df['FL']*2 + wr_proj_df['REC']
te_proj_df['Proj FPTS'] = te_proj_df['YDS']/10 + te_proj_df['TDS']*6 - te_proj_df['FL']*2 + te_proj_df['REC']
all_proj_df = pd.concat([qb_proj_df[['Player','Proj FPTS']], rb_proj_df[['Player','Proj FPTS']], wr_proj_df[['Player','Proj FPTS']], te_proj_df[['Player','Proj FPTS']]])

In [985]:
all_proj_df.head()

,Player,Proj FPTS
0,Josh Allen BUF,387.538
1,Patrick Mahomes II KC,361.360
2,Justin Herbert LAC,342.732
3,Lamar Jackson BAL,339.010
4,Kyler Murray ARI,338.784


In [565]:
#all_proj_df = all_proj_df.rename(columns={"FPTS":'Proj FPTS'})

In [6]:
qb_df = pd.read_html(qb_url, header=1)[0]
rb_df = pd.read_html(rb_url, header=1)[0]
wr_df = pd.read_html(wr_url, header=1)[0]
te_df = pd.read_html(te_url, header=1)[0]
dst_df = pd.read_html(dst_url, header=0)[0]

adp_df = pd.read_html(adp_url, header=0)[0]

In [147]:
qb_df.head()

,Rank,Player,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT.1,YDS.1,TD.1,FL,G,FPTS,FPTS/G,ROST,Position
0,1,Josh Allen (BUF),409,646,63.3,4407,6.8,36,15,26,122,763,6,3,17,417.7,24.6,100.0%,QB
1,2,Justin Herbert (LAC),443,672,65.9,5014,7.5,38,15,31,63,302,3,1,17,395.6,23.3,99.9%,QB
2,3,Tom Brady (TB),485,719,67.5,5316,7.4,43,12,22,28,81,2,3,17,386.7,22.7,96.9%,QB
3,4,Patrick Mahomes II (KC),436,658,66.3,4828,7.3,37,13,28,66,381,2,4,17,374.2,22.0,99.9%,QB
4,5,Matthew Stafford (LAR),404,601,67.2,4886,8.1,41,17,30,32,43,0,2,17,346.8,20.4,91.1%,QB


In [7]:
adp_df['Player Team (Bye)'] = adp_df['Player Team (Bye)'].str.replace("\(.*\)", "")
adp_df

,Rank,Player Team (Bye),POS,Yahoo,Fantrax,FFC,Sleeper,AVG
0,1,Jonathan Taylor IND,RB1,1.0,1.0,NaN,1.0,1.0
1,2,Christian McCaffrey CAR,RB2,2.0,3.0,NaN,2.0,2.3
2,3,Austin Ekeler LAC,RB3,3.0,2.0,NaN,4.0,3.0
3,4,Cooper Kupp LAR,WR1,4.0,4.0,NaN,5.0,4.3
4,5,Derrick Henry TEN,RB4,6.0,5.0,NaN,3.0,4.7
...,...,...,...,...,...,...,...,...
435,436,Jordan Love GB,QB49,NaN,427.0,NaN,NaN,427.0
436,437,Harrison Bryant CLE,TE54,NaN,428.0,NaN,NaN,428.0
437,438,Geoff Swaim TEN,TE55,NaN,430.0,NaN,NaN,430.0
438,439,Pharaoh Brown HOU,TE56,NaN,431.0,NaN,NaN,431.0


In [410]:
adp_df[adp_df['POS'] == 'QB2']

,Rank,Player Team (Bye),POS,Yahoo,Fantrax,FFC,Sleeper,AVG
33,34,Patrick Mahomes II KC,QB2,34.0,40.0,36.0,32.0,35.5


In [8]:
positions = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'R/W/T': 2}
positions_v1 = {'QB': 1, 'RB': 4, 'WR': 2, 'TE': 1, 'DST': 1}
positions_v2 = {'QB': 1, 'RB': 3, 'WR': 3, 'TE': 1, 'DST': 1}
positions_v3 = {'QB': 1, 'RB': 2, 'WR': 4, 'TE': 1, 'DST': 1}

In [132]:
qb_df['Position'] = 'QB'
rb_df['Position'] = 'RB'
wr_df['Position'] = 'WR'
te_df['Position'] = 'TE'
dst_df['Position'] = 'DST'

all_df = pd.concat([qb_df[['Player','Position','FPTS','FPTS/G']], rb_df[['Player','Position','FPTS','FPTS/G']], wr_df[['Player','Position','FPTS','FPTS/G']], te_df[['Player','Position','FPTS','FPTS/G']], dst_df[['Player','Position','FPTS','FPTS/G']]], axis=0)

In [133]:
all_df['Player'] = all_df['Player'].str.replace("\(", "")
all_df['Player'] = all_df['Player'].str.replace("\)", "")

all_df.head()

,Player,Position,FPTS,FPTS/G
0,Josh Allen BUF,QB,417.7,24.6
1,Justin Herbert LAC,QB,395.6,23.3
2,Tom Brady TB,QB,386.7,22.7
3,Patrick Mahomes II KC,QB,374.2,22.0
4,Matthew Stafford LAR,QB,346.8,20.4


In [134]:
all_df['RWT'] = all_df['Position'].isin(['RB','WR','TE']).astype(int)

In [12]:
all_df[all_df['Player']=='Josh Allen BUF']

,Player,Position,FPTS,FPTS/G,RWT
0,Josh Allen BUF,QB,417.7,24.6,0


In [416]:
adp_df['Player'] = adp_df["Player Team (Bye)"]
adp_df.head()

,Rank,Player Team (Bye),POS,Yahoo,Fantrax,FFC,Sleeper,AVG,Player
0,1,Jonathan Taylor IND,RB1,1.0,1.0,1.0,1.0,1.0,Jonathan Taylor IND
1,2,Austin Ekeler LAC,RB2,3.0,2.0,3.0,4.0,3.0,Austin Ekeler LAC
2,3,Cooper Kupp LAR,WR1,2.0,3.0,4.0,5.0,3.5,Cooper Kupp LAR
3,4,Derrick Henry TEN,RB3,5.0,5.0,2.0,3.0,3.8,Derrick Henry TEN
4,5,Christian McCaffrey CAR,RB4,4.0,4.0,6.0,2.0,4.0,Christian McCaffrey CAR


In [417]:
all_df.dtypes

Player       object
Position     object
FPTS        float64
FPTS/G      float64
RWT           int64
dtype: object

In [418]:
adp_df.dtypes

Rank                   int64
Player Team (Bye)     object
POS                   object
Yahoo                float64
Fantrax              float64
FFC                  float64
Sleeper              float64
AVG                  float64
Player                object
dtype: object

In [13]:
all_df.Player = all_df.Player.astype(str)
adp_df["Player Team (Bye)"] = adp_df["Player Team (Bye)"].astype(str)
adp_df.Player = adp_df.Player.astype(str)

AttributeError: 'DataFrame' object has no attribute 'Player'

In [420]:
adp_df.Player = adp_df.Player.str.encode('utf-8')

In [421]:
all_df.Player

0               Josh Allen BUF
1           Justin Herbert LAC
2                 Tom Brady TB
3        Patrick Mahomes II KC
4         Matthew Stafford LAR
                ...           
27        Las Vegas Raiders LV
28         Atlanta Falcons ATL
29    Jacksonville Jaguars JAC
30           New York Jets NYJ
31           Detroit Lions DET
Name: Player, Length: 1148, dtype: object

In [422]:
adp_df.Player

0          b'Jonathan Taylor IND '
1            b'Austin Ekeler LAC '
2              b'Cooper Kupp LAR '
3            b'Derrick Henry TEN '
4      b'Christian McCaffrey CAR '
                  ...             
401           b'Dee Eskridge SEA '
402      b'Donald Parham Jr. LAC '
403             b'Jordan Love GB '
404            b'Matt Corral CAR '
405          b'Pharaoh Brown HOU '
Name: Player, Length: 406, dtype: object

In [135]:
adp_df['Player'] = adp_df["Player Team (Bye)"].str.strip()

In [802]:
all_proj_df.shape

(676, 2)

In [801]:
adp_df.shape

(406, 9)

In [136]:
players = all_df.merge(adp_df, left_on = 'Player', right_on = "Player", how='right')

In [137]:
players = players.merge(all_proj_df, how='inner')

In [138]:
players['X_PTS_1'] = players['Proj FPTS']*1/2 + players['FPTS']*1/2

In [142]:
players['Drafted'] = 0

In [139]:
players.shape

(350, 15)

In [140]:
players[players['Player']=='Garrett Wilson NYJ']

,Player,Position,FPTS,FPTS/G,RWT,Rank,Player Team (Bye),POS,Yahoo,Fantrax,FFC,Sleeper,AVG,Proj FPTS,X_PTS_1
118,Garrett Wilson NYJ,WR,0.0,0.0,1.0,122,Garrett Wilson NYJ,WR48,166.0,113.0,NaN,107.0,128.7,149.73,74.865


In [20]:
players.tail(100)

,Player,Position,FPTS,FPTS/G,RWT,Rank,Player Team (Bye),POS,Yahoo,Fantrax,FFC,Sleeper,AVG,Proj FPTS
250,Phillip Lindsay IND,RB,43.4,3.1,1.0,304,Phillip Lindsay IND,RB86,NaN,269.0,NaN,NaN,269.0,36.960
251,Sterling Shepard NYG,WR,59.7,8.5,1.0,307,Sterling Shepard NYG,WR100,NaN,271.0,NaN,NaN,271.0,106.420
252,Anthony McFarland Jr. PIT,RB,1.9,0.6,1.0,310,Anthony McFarland Jr. PIT,RB87,NaN,273.0,NaN,NaN,273.0,22.320
253,John FitzPatrick ATL,TE,0.0,0.0,1.0,312,John FitzPatrick ATL,TE37,NaN,NaN,NaN,274.0,274.0,20.070
254,Jalen Guyton LAC,WR,81.7,5.1,1.0,314,Jalen Guyton LAC,WR102,NaN,275.0,NaN,NaN,275.0,56.970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Joe Flacco NYJ,QB,23.8,11.9,0.0,435,Joe Flacco NYJ,QB48,NaN,426.0,NaN,NaN,426.0,26.650
346,Jordan Love GB,QB,24.2,4.0,0.0,436,Jordan Love GB,QB49,NaN,427.0,NaN,NaN,427.0,10.754
347,Harrison Bryant CLE,TE,51.8,3.5,1.0,437,Harrison Bryant CLE,TE54,NaN,428.0,NaN,NaN,428.0,81.730
348,Geoff Swaim TEN,TE,54.5,3.4,1.0,438,Geoff Swaim TEN,TE55,NaN,430.0,NaN,NaN,430.0,68.640


In [143]:
player_arr = np.asarray(players[['Position','FPTS','FPTS/G','Proj FPTS','RWT','AVG','Player','X_PTS_1','Drafted']])
player_arr

array([['RB', 353.1, 20.8, ..., 'Jonathan Taylor IND', 343.625, 0],
       ['RB', 109.0, 15.6, ..., 'Christian McCaffrey CAR', 223.415, 0],
       ['RB', 308.8, 19.3, ..., 'Austin Ekeler LAC', 316.99, 0],
       ...,
       ['TE', 51.8, 3.5, ..., 'Harrison Bryant CLE', 62.365, 0],
       ['TE', 54.5, 3.4, ..., 'Geoff Swaim TEN', 59.295, 0],
       ['TE', 26.6, 2.0, ..., 'Pharaoh Brown HOU', 39.870000000000005, 0]],
      dtype=object)

In [144]:
positions = np.asarray(pd.get_dummies(players['Position']))
positions

array([[0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       ...,
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0]], dtype=uint8)

In [993]:
len(player_arr)

333

In [995]:
len(positions)

333

In [126]:
draft_1 = np.asarray([1,24,25,48,49,72,73,96])
draft_2 = np.asarray([draft_1[0]+1, draft_1[1]-1, draft_1[2]+1, draft_1[3]-1, draft_1[4]+1, draft_1[5]-1, draft_1[6]+1, draft_1[7]-1])
draft_3 = np.asarray([draft_2[0]+1, draft_2[1]-1, draft_2[2]+1, draft_2[3]-1, draft_2[4]+1, draft_2[5]-1, draft_2[6]+1, draft_2[7]-1])
draft_4 = np.asarray([draft_3[0]+1, draft_3[1]-1, draft_3[2]+1, draft_3[3]-1, draft_3[4]+1, draft_3[5]-1, draft_3[6]+1, draft_3[7]-1])
draft_5 = np.asarray([draft_4[0]+1, draft_4[1]-1, draft_4[2]+1, draft_4[3]-1, draft_4[4]+1, draft_4[5]-1, draft_4[6]+1, draft_4[7]-1])
draft_6 = np.asarray([draft_5[0]+1, draft_5[1]-1, draft_5[2]+1, draft_5[3]-1, draft_5[4]+1, draft_5[5]-1, draft_5[6]+1, draft_5[7]-1])
draft_7 = np.asarray([draft_6[0]+1, draft_6[1]-1, draft_6[2]+1, draft_6[3]-1, draft_6[4]+1, draft_6[5]-1, draft_6[6]+1, draft_6[7]-1])
draft_8 = np.asarray([draft_7[0]+1, draft_7[1]-1, draft_7[2]+1, draft_7[3]-1, draft_7[4]+1, draft_7[5]-1, draft_7[6]+1, draft_7[7]-1])
draft_9 = np.asarray([draft_8[0]+1, draft_8[1]-1, draft_8[2]+1, draft_8[3]-1, draft_8[4]+1, draft_8[5]-1, draft_8[6]+1, draft_8[7]-1])
draft_10 = np.asarray([draft_9[0]+1, draft_9[1]-1, draft_9[2]+1, draft_9[3]-1, draft_9[4]+1, draft_9[5]-1, draft_9[6]+1, draft_9[7]-1])
draft_11 = np.asarray([draft_10[0]+1, draft_10[1]-1, draft_10[2]+1, draft_10[3]-1, draft_10[4]+1, draft_10[5]-1, draft_10[6]+1, draft_10[7]-1])
draft_12 = np.asarray([draft_11[0]+1, draft_11[1]-1, draft_11[2]+1, draft_11[3]-1, draft_11[4]+1, draft_11[5]-1, draft_11[6]+1, draft_11[7]-1])





In [26]:
from gurobi import *

In [435]:
draft_order[0]

1

In [996]:
# 1ST OVERALL PICK

m = Model()

draft_order = draft_1

# player dummy variable
x = m.addVars(len(player_arr), vtype=GRB.BINARY)

# draft round variable
y = m.addVars(len(draft_order), len(player_arr), vtype=GRB.BINARY)


# constraint: can't pick the same player twice
#for k in range(len(draft_order)):
 #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
for i in range(len(player_arr)):
    m.addConstr(sum(y[k,i] for k in range(len(draft_order))) <= 1)
    
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] for i in range(len(player_arr))) == 1) # constraint -- each round draft one player
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] * player_arr[i,5] for i in range(len(player_arr))) >= draft_order[k]) #ADP constraint
    #m.addConstr(y[k,i] == x[i])

# starting lineup has 8 players -- constraint
#m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

# constraint: maximum of 7 flex eligible players
#m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

#constraint: total of 8 players
m.addConstr(sum(y[k,i] for k in range(len(draft_order)) for i in range(len(player_arr))) == 8)



# QB constraint: 1 QB in lineup
m.addConstr(sum(y[k,i] * positions[i,0] for k in range(len(draft_order)) for i in range(len(player_arr))) == 1)
# # RB constraint: minimum of 2 RBs in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)
# # TE constraint: minimum of 1 TE in lineup, maximum of 3
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 1)
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 3)
# # WR constraint: minimum of 2 WR in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)

    
m.setObjective(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))), GRB.MAXIMIZE)


m.update()
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 357 rows, 2997 columns and 15624 nonzeros
Model fingerprint: 0x5e840bf1
Variable types: 0 continuous, 2997 integer (2997 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [2e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective 993.4740000
Presolve removed 78 rows and 1425 columns
Presolve time: 0.04s
Presolved: 279 rows, 1572 columns, 6267 nonzeros
Found heuristic solution: objective 1805.2180000
Variable types: 0 continuous, 1572 integer (1572 binary)

Root relaxation: objective 1.854748e+03, 400 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1854.7480000 1854.74800  0.00%     - 

In [997]:
players_x = np.asarray(m.x)
indices = np.where(players_x > 0.5)[0]

draft_opt = np.array([indices[0] - len(player_arr)*1, indices[1] - len(player_arr)*2, 
                      indices[2] - len(player_arr)*3, indices[3] - len(player_arr)*4, 
                      indices[4] - len(player_arr)*5, indices[5] - len(player_arr)*6, 
                      indices[6] - len(player_arr)*7, indices[7] - len(player_arr)*8]
)

In [998]:
for i in range(len(draft_opt)):
    print(player_arr[draft_opt[i]])

['RB' 353.1 20.8 313.53 1.0 1.0 'Jonathan Taylor IND']
['QB' 417.7 24.6 387.538 0.0 22.8 'Josh Allen BUF']
['TE' 247.6 14.6 218.33 1.0 23.3 'Mark Andrews BAL']
['WR' 208.6 13.0 193.72999999999996 1.0 48.3 'Mike Williams LAC']
['WR' 191.0 11.2 194.94 1.0 44.5 'DJ Moore CAR']
['WR' 186.8 11.7 194.25999999999996 1.0 62.3 'Brandin Cooks HOU']
['RB' 104.2 8.7 181.37 1.0 70.8 'Miles Sanders PHI']
['WR' 204.9 12.8 171.05 1.0 99.8 'Tyler Lockett SEA']


In [816]:
# 2ND OVERALL PICK

m = Model()

draft_order = draft_2

# player dummy variable
x = m.addVars(len(player_arr), vtype=GRB.BINARY)

# draft round variable
y = m.addVars(len(draft_order), len(player_arr), vtype=GRB.BINARY)


# constraint: can't pick the same player twice
#for k in range(len(draft_order)):
 #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
for i in range(len(player_arr)):
    m.addConstr(sum(y[k,i] for k in range(len(draft_order))) <= 1)
    
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] for i in range(len(player_arr))) == 1) # constraint -- each round draft one player
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] * player_arr[i,5] for i in range(len(player_arr))) >= draft_order[k]) #ADP constraint
    #m.addConstr(y[k,i] == x[i])

# starting lineup has 8 players -- constraint
#m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

# constraint: maximum of 7 flex eligible players
#m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

#constraint: total of 8 players
m.addConstr(sum(y[k,i] for k in range(len(draft_order)) for i in range(len(player_arr))) == 8)



# QB constraint: 1 QB in lineup
m.addConstr(sum(y[k,i] * positions[i,0] for k in range(len(draft_order)) for i in range(len(player_arr))) == 1)
# # RB constraint: minimum of 2 RBs in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)
# # TE constraint: minimum of 1 TE in lineup, maximum of 3
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 1)
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 3)
# # WR constraint: minimum of 2 WR in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)

    
m.setObjective(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))), GRB.MAXIMIZE)


m.update()
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 703 rows, 6111 columns and 23928 nonzeros
Model fingerprint: 0x4af5e353
Variable types: 0 continuous, 6111 integer (6111 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e-01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective 596.8180000
Presolve removed 435 rows and 4909 columns
Presolve time: 0.05s
Presolved: 268 rows, 1202 columns, 4784 nonzeros
Found heuristic solution: objective 1839.3580000
Variable types: 0 continuous, 1202 integer (1202 binary)

Root relaxation: objective 1.851294e+03, 412 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1851.2940000 1851.29400  0.00%     -

In [818]:
players_x = np.asarray(m.x)
indices = np.where(players_x > 0.5)[0]

draft_opt = np.array([indices[0] - len(player_arr)*1, indices[1] - len(player_arr)*2, 
                      indices[2] - len(player_arr)*3, indices[3] - len(player_arr)*4, 
                      indices[4] - len(player_arr)*5, indices[5] - len(player_arr)*6, 
                      indices[6] - len(player_arr)*7, indices[7] - len(player_arr)*8]
)

for i in range(len(draft_opt)):
    print(player_arr[draft_opt[i]])

['WR' 367.0 21.6 296.46999999999997 1.0 3.5 'Cooper Kupp LAR']
['RB' 211.2 16.2 245.95 1.0 19.5 'Alvin Kamara NO']
['TE' 247.6 14.6 218.33 1.0 23.3 'Mark Andrews BAL']
['RB' 0.0 0.0 191.51 1.0 54.8 'J.K. Dobbins BAL']
['WR' 191.0 11.2 194.94 1.0 44.5 'DJ Moore CAR']
['QB' 310.5 22.2 338.784 0.0 59.5 'Kyler Murray ARI']
['WR' 186.8 11.7 194.25999999999996 1.0 62.3 'Brandin Cooks HOU']
['WR' 204.9 12.8 171.05 1.0 99.8 'Tyler Lockett SEA']


In [819]:
# 3RD OVERALL PICK

m = Model()

draft_order = draft_3

# player dummy variable
x = m.addVars(len(player_arr), vtype=GRB.BINARY)

# draft round variable
y = m.addVars(len(draft_order), len(player_arr), vtype=GRB.BINARY)


# constraint: can't pick the same player twice
#for k in range(len(draft_order)):
 #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
for i in range(len(player_arr)):
    m.addConstr(sum(y[k,i] for k in range(len(draft_order))) <= 1)
    
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] for i in range(len(player_arr))) == 1) # constraint -- each round draft one player
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] * player_arr[i,5] for i in range(len(player_arr))) >= draft_order[k]) #ADP constraint
    #m.addConstr(y[k,i] == x[i])

# starting lineup has 8 players -- constraint
#m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

# constraint: maximum of 7 flex eligible players
#m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

#constraint: total of 8 players
m.addConstr(sum(y[k,i] for k in range(len(draft_order)) for i in range(len(player_arr))) == 8)



# QB constraint: 1 QB in lineup
m.addConstr(sum(y[k,i] * positions[i,0] for k in range(len(draft_order)) for i in range(len(player_arr))) == 1)
# # RB constraint: minimum of 2 RBs in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)
# # TE constraint: minimum of 1 TE in lineup, maximum of 3
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 1)
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 3)
# # WR constraint: minimum of 2 WR in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)

    
m.setObjective(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))), GRB.MAXIMIZE)


m.update()
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 703 rows, 6111 columns and 23928 nonzeros
Model fingerprint: 0xb58b4efd
Variable types: 0 continuous, 6111 integer (6111 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e-01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective 596.8180000
Presolve removed 513 rows and 5292 columns
Presolve time: 0.06s
Presolved: 190 rows, 819 columns, 3247 nonzeros
Found heuristic solution: objective 1835.8580000
Variable types: 0 continuous, 819 integer (819 binary)

Root relaxation: objective 1.851084e+03, 301 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1851.0840000 1851.08400  0.00%     -   

In [820]:
players_x = np.asarray(m.x)
indices = np.where(players_x > 0.5)[0]

draft_opt = np.array([indices[0] - len(player_arr)*1, indices[1] - len(player_arr)*2, 
                      indices[2] - len(player_arr)*3, indices[3] - len(player_arr)*4, 
                      indices[4] - len(player_arr)*5, indices[5] - len(player_arr)*6, 
                      indices[6] - len(player_arr)*7, indices[7] - len(player_arr)*8]
)

for i in range(len(draft_opt)):
    print(player_arr[draft_opt[i]])

['WR' 367.0 21.6 296.46999999999997 1.0 3.5 'Cooper Kupp LAR']
['RB' 211.2 16.2 245.95 1.0 19.5 'Alvin Kamara NO']
['TE' 247.6 14.6 218.33 1.0 23.3 'Mark Andrews BAL']
['WR' 208.6 13.0 193.72999999999996 1.0 48.3 'Mike Williams LAC']
['WR' 191.0 11.2 194.94 1.0 44.5 'DJ Moore CAR']
['WR' 186.8 11.7 194.25999999999996 1.0 62.3 'Brandin Cooks HOU']
['RB' 104.2 8.7 181.37 1.0 70.8 'Miles Sanders PHI']
['QB' 386.7 22.7 326.034 0.0 81.5 'Tom Brady TB']


In [821]:
# 4TH OVERALL PICK

m = Model()

draft_order = draft_4

# player dummy variable
x = m.addVars(len(player_arr), vtype=GRB.BINARY)

# draft round variable
y = m.addVars(len(draft_order), len(player_arr), vtype=GRB.BINARY)


# constraint: can't pick the same player twice
#for k in range(len(draft_order)):
 #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
for i in range(len(player_arr)):
    m.addConstr(sum(y[k,i] for k in range(len(draft_order))) <= 1)
    
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] for i in range(len(player_arr))) == 1) # constraint -- each round draft one player
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] * player_arr[i,5] for i in range(len(player_arr))) >= draft_order[k]) #ADP constraint
    #m.addConstr(y[k,i] == x[i])

# starting lineup has 8 players -- constraint
#m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

# constraint: maximum of 7 flex eligible players
#m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

#constraint: total of 8 players
m.addConstr(sum(y[k,i] for k in range(len(draft_order)) for i in range(len(player_arr))) == 8)



# QB constraint: 1 QB in lineup
m.addConstr(sum(y[k,i] * positions[i,0] for k in range(len(draft_order)) for i in range(len(player_arr))) == 1)
# # RB constraint: minimum of 2 RBs in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)
# # TE constraint: minimum of 1 TE in lineup, maximum of 3
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 1)
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 3)
# # WR constraint: minimum of 2 WR in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)

    
m.setObjective(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))), GRB.MAXIMIZE)


m.update()
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 703 rows, 6111 columns and 23928 nonzeros
Model fingerprint: 0xf34947aa
Variable types: 0 continuous, 6111 integer (6111 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e-01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective 596.8180000
Presolve removed 558 rows and 5777 columns
Presolve time: 0.06s
Presolved: 145 rows, 334 columns, 1301 nonzeros
Found heuristic solution: objective 1824.0540000
Variable types: 0 continuous, 334 integer (334 binary)

Root relaxation: objective 1.836414e+03, 194 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1836.4140000 1836.41400  0.00%     -   

In [822]:
players_x = np.asarray(m.x)
indices = np.where(players_x > 0.5)[0]

draft_opt = np.array([indices[0] - len(player_arr)*1, indices[1] - len(player_arr)*2, 
                      indices[2] - len(player_arr)*3, indices[3] - len(player_arr)*4, 
                      indices[4] - len(player_arr)*5, indices[5] - len(player_arr)*6, 
                      indices[6] - len(player_arr)*7, indices[7] - len(player_arr)*8]
)

for i in range(len(draft_opt)):
    print(player_arr[draft_opt[i]])

['RB' 109.0 15.6 292.36 1.0 4.0 'Christian McCaffrey CAR']
['RB' 211.2 16.2 245.95 1.0 19.5 'Alvin Kamara NO']
['RB' 239.2 15.9 221.36000000000004 1.0 31.3 'James Conner ARI']
['WR' 208.6 13.0 193.72999999999996 1.0 48.3 'Mike Williams LAC']
['WR' 191.0 11.2 194.94 1.0 44.5 'DJ Moore CAR']
['WR' 186.8 11.7 194.25999999999996 1.0 62.3 'Brandin Cooks HOU']
['TE' 169.8 10.0 167.78 1.0 65.0 'Dalton Schultz DAL']
['QB' 386.7 22.7 326.034 0.0 81.5 'Tom Brady TB']


In [661]:
players[players['AVG']<=10]

,Player,Position,FPTS,FPTS/G,RWT,Rank,Player Team (Bye),POS,Yahoo,Fantrax,FFC,Sleeper,AVG,Proj FPTS
47,Jonathan Taylor IND,RB,353.1,20.8,1,1,Jonathan Taylor IND,RB1,1.0,1.0,1.0,1.0,1.0,313.53
48,Austin Ekeler LAC,RB,308.8,19.3,1,2,Austin Ekeler LAC,RB2,3.0,2.0,3.0,4.0,3.0,286.59
49,Joe Mixon CIN,RB,266.9,16.7,1,10,Joe Mixon CIN,RB7,10.0,11.0,8.0,10.0,9.8,247.34
50,Najee Harris PIT,RB,263.7,15.5,1,7,Najee Harris PIT,RB5,9.0,7.0,5.0,9.0,7.5,241.77
61,Dalvin Cook MIN,RB,189.3,14.6,1,8,Dalvin Cook MIN,RB6,7.0,8.0,9.0,6.0,7.5,244.27
62,Derrick Henry TEN,RB,184.3,23.0,1,4,Derrick Henry TEN,RB3,5.0,5.0,2.0,3.0,3.8,280.03
83,Christian McCaffrey CAR,RB,109.0,15.6,1,5,Christian McCaffrey CAR,RB4,4.0,4.0,6.0,2.0,4.0,292.36
150,Cooper Kupp LAR,WR,367.0,21.6,1,3,Cooper Kupp LAR,WR1,2.0,3.0,4.0,5.0,3.5,296.47
153,Justin Jefferson MIN,WR,276.4,16.3,1,6,Justin Jefferson MIN,WR2,6.0,6.0,7.0,7.0,6.5,260.28
154,Ja'Marr Chase CIN,WR,264.1,15.5,1,9,Ja'Marr Chase CIN,WR3,8.0,9.0,11.0,8.0,9.0,255.59


In [102]:
player_arr[player_arr[:,6]=='Darren Waller LV']

array([['TE', 106.0, 9.6, 157.58999999999997, 1.0, 45.7,
        'Darren Waller LV', 131.795, 0]], dtype=object)

In [248]:
players[players['Player'] == 'Miles Sanders PHI'].index.values[0]

72

In [145]:
test_player_arr = player_arr.copy()

In [57]:
def draft(player, pick):
    idx = players[players['Player'] == player].index.values[0]
    adp = 5
    test_player_arr[idx,adp] = pick
    test_player_arr[idx,-1] = 1

In [71]:
def players_avail(pick):
    for idx in range(len(test_player_arr)):
        if test_player_arr[idx,-1] == 0 and test_player_arr[idx,5] < pick:
            test_player_arr[idx,5] = pick + 0.1
            
    return test_player_arr[(test_player_arr[:,5]>=pick) & (test_player_arr[:,5]<=pick+5)]

In [263]:
draft("Jonathan Taylor IND", 1)
draft("Christian McCaffrey CAR", 2)
draft("Cooper Kupp LAR", 3)
draft("Justin Jefferson MIN", 4)
draft("Austin Ekeler LAC", 5)
draft("Ja'Marr Chase CIN", 6)
draft("Najee Harris PIT", 7)
draft("Derrick Henry TEN", 8)
draft("Dalvin Cook MIN", 9)
draft("Stefon Diggs BUF", 10)
draft("Alvin Kamara NO", 11)
draft("Davante Adams LV", 12)
draft("Joe Mixon CIN", 13)
draft("Travis Kelce KC", 14)
draft("D'Andre Swift DET", 15)
draft("Saquon Barkley NYG", 16)
draft("Deebo Samuel SF", 17)
draft("DJ Moore CAR", 18)
draft("CeeDee Lamb DAL", 19)
draft("Aaron Jones GB", 20)
draft("Mark Andrews BAL", 21)
draft("Tyreek Hill MIA", 22)
draft("James Conner ARI", 23)
draft("Keenan Allen LAC", 24)
draft("Mike Evans TB", 25)
draft("A.J. Brown PHI", 26)
draft("Leonard Fournette TB", 27)
draft("Javonte Williams DEN", 28)
draft("Nick Chubb CLE", 29)
draft("Cam Akers LAR", 30)
draft("Travis Etienne Jr. JAC", 31)
draft("Michael Pittman Jr. IND", 32)
draft("Tee Higgins CIN", 33)
draft("Kyle Pitts ATL", 34)
draft("Josh Allen BUF", 35)
draft("Terry McLaurin WAS", 36)
draft("Mike Williams LAC", 37)
draft("David Montgomery CHI", 38)
draft("Diontae Johnson PIT", 39)
draft("Courtland Sutton DEN", 40)
draft("Patrick Mahomes II KC", 41)
draft("Brandin Cooks HOU", 42)
draft("DK Metcalf SEA", 43)
draft("Jaylen Waddle MIA", 44)
draft("Ezekiel Elliott DAL", 45)
draft("Breece Hall NYJ", 46)
draft("J.K. Dobbins BAL", 47)
draft("Amon-Ra St. Brown DET", 48)
draft("Elijah Mitchell SF", 49)
draft("Jerry Jeudy DEN", 50)
draft("Justin Herbert LAC", 51)
draft("Kyler Murray ARI", 52)
draft("Lamar Jackson BAL", 53)
draft("Josh Jacobs LV", 54)
draft("Darnell Mooney CHI", 55)
draft("Chris Godwin TB", 56)
draft("Marquise Brown ARI", 57)
draft("Amari Cooper CLE", 58)
draft("Gabriel Davis BUF", 59)
draft("George Kittle SF", 60)
draft("Jalen Hurts PHI", 61)
draft("Hunter Renfrow LV", 62)
draft("Clyde Edwards-Helaire KC", 63)
draft("Dameon Pierce HOU", 64)
draft("Devin Singletary BUF", 65)
draft("JuJu Smith-Schuster KC", 66)
draft("Darren Waller LV", 67)
draft("Allen Robinson II LAR", 68)
draft("Christian Kirk JAC", 69)
draft("Adam Thielen MIN", 70)
draft("Michael Thomas NO", 71)
draft("Dalton Schultz DAL", 72)
draft("Drake London ATL", 73)
draft("T.J. Hockenson DET", 74)
draft("Rashod Bateman BAL", 75)
draft("DeVonta Smith PHI", 76)
draft("AJ Dillon GB", 77)
draft("Antonio Gibson WAS", 78)
draft("Dallas Goedert PHI", 79)
draft("Zach Ertz ARI", 80)
draft("Aaron Rodgers GB", 81)
draft("Dak Prescott DAL", 82)
draft("Miles Sanders PHI", 83)
draft("Kareem Hunt CLE", 84)
draft("Allen Lazard GB", 85)
draft("Tyler Lockett SEA", 86)
draft("Brandon Aiyuk SF", 87)
draft("Elijah Moore NYJ", 88)
draft("Chase Claypool PIT", 89)
draft("Trey Lance SF", 90)
draft("Tom Brady TB", 91)
draft("Kadarius Toney NYG", 92)
draft("Chase Edmonds MIA", 93)
draft("Rhamondre Stevenson NE", 94)
draft("Russell Wilson DEN", 95)
draft("Joe Burrow CIN", 96)

In [253]:
# players_avail(draft_order[0])
# players_avail(draft_order[1])
# players_avail(draft_order[2])
# players_avail(draft_order[3])
# players_avail(draft_order[4])
# players_avail(draft_order[5])
# players_avail(draft_order[6])
players_avail(draft_order[7])

array([['QB', 328.1, 20.5, 310.9, 0.0, 86.1, 'Joe Burrow CIN', 319.5, 0],
       ['RB', 201.1, 13.4, 188.85999999999999, 1.0, 86.1,
        'Damien Harris NE', 194.98, 0],
       ['QB', 248.7, 17.8, 297.026, 0.0, 86.1, 'Russell Wilson DEN',
        272.863, 0],
       ['QB', 386.7, 22.7, 325.082, 0.0, 86.1, 'Tom Brady TB',
        355.89099999999996, 0],
       ['RB', 208.6, 13.0, 185.6, 1.0, 86.1, 'Cordarrelle Patterson ATL',
        197.1, 0],
       ['WR', 116.7, 10.6, 204.24, 1.0, 86.1, 'Elijah Moore NYJ', 160.47,
        0],
       ['RB', 118.7, 11.9, 170.22, 1.0, 86.1, 'Rashaad Penny SEA',
        144.46, 0],
       ['RB', 121.8, 10.2, 174.13, 1.0, 86.7, 'Chase Edmonds MIA',
        147.965, 0],
       ['RB', 143.1, 9.5, 193.39, 1.0, 87.3, 'Tony Pollard DAL', 168.245,
        0],
       ['TE', 139.6, 9.3, 145.1, 1.0, 90.7, 'Dawson Knox BUF', 142.35, 0]],
      dtype=object)

In [265]:
draft_opt()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 374 rows, 3150 columns and 16424 nonzeros
Model fingerprint: 0xa0ec4dfe
Variable types: 0 continuous, 3150 integer (3150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [3e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective 923.5320000
Presolve removed 204 rows and 2195 columns
Presolve time: 0.05s
Presolved: 170 rows, 955 columns, 3794 nonzeros
Found heuristic solution: objective 1957.3520000
Variable types: 0 continuous, 955 integer (955 binary)

Root relaxation: objective 2.013112e+03, 213 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2013.1120000 2013.11200  0.00%     -   

In [264]:
# 5TH OVERALL PICK
def draft_opt():


    m = Model()

    draft_order = draft_11
    draft_len = 8

    # player dummy variable
    x = m.addVars(len(test_player_arr), vtype=GRB.BINARY)

    # draft round variable
    y = m.addVars(draft_len, len(test_player_arr), vtype=GRB.BINARY)

    # ADD WHO YOU DRAFTED AS CONSTRAINTS:
    # y[rd,idx] == 1
#     m.addConstr(y[0,16]==1)
#     m.addConstr(y[1,12]==1)
#     m.addConstr(y[2,21]==1)
#     m.addConstr(y[3,34]==1)
#     m.addConstr(y[4,66]==1)
#     m.addConstr(y[5,82]==1)
#     m.addConstr(y[6,72]==1)
#     m.addConstr(y[7,72]==1)


    # constraint: can't pick the same player twice
    #for k in range(len(draft_order)):
     #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
    for i in range(len(test_player_arr)):
        m.addConstr(sum(y[k,i] for k in range(draft_len)) <= 1)


    for k in range(draft_len):
        m.addConstr(sum(y[k,i] for i in range(len(test_player_arr))) == 1) # constraint -- each round draft one player

    for k in range(draft_len):
        m.addConstr(sum(y[k,i] * test_player_arr[i,5] for i in range(len(test_player_arr))) >= draft_order[k]) #ADP constraint
        #m.addConstr(y[k,i] == x[i])

    # starting lineup has 8 players -- constraint
    #m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

    # constraint: maximum of 7 flex eligible players
    #m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

    #constraint: total of 8 players
    m.addConstr(sum(y[k,i] for k in range(draft_len) for i in range(len(test_player_arr))) == 8)



    # QB constraint: 1 QB in lineup
    m.addConstr(sum(y[k,i] * positions[i,0] for k in range(draft_len) for i in range(len(test_player_arr))) == 1)
    # # RB constraint: minimum of 2 RBs in lineup, maximum of 4
    m.addConstr(sum(y[k,i] * positions[i,1] for k in range(draft_len) for i in range(len(test_player_arr))) >= 2)
    m.addConstr(sum(y[k,i] * positions[i,1] for k in range(draft_len) for i in range(len(test_player_arr))) <= 4)
    # # TE constraint: minimum of 1 TE in lineup, maximum of 3
    m.addConstr(sum(y[k,i] * positions[i,2] for k in range(draft_len) for i in range(len(test_player_arr))) >= 1)
    m.addConstr(sum(y[k,i] * positions[i,2] for k in range(draft_len) for i in range(len(test_player_arr))) <= 3)
    # # WR constraint: minimum of 2 WR in lineup, maximum of 4
    m.addConstr(sum(y[k,i] * positions[i,3] for k in range(draft_len) for i in range(len(test_player_arr))) >= 2)
    m.addConstr(sum(y[k,i] * positions[i,3] for k in range(draft_len) for i in range(len(test_player_arr))) <= 4)


    m.setObjective(sum(y[k,i] * test_player_arr[i,3] for k in range(draft_len) for i in range(len(player_arr))), GRB.MAXIMIZE)


    m.update()
    m.optimize()
    

    players_x = np.asarray(m.x)
    indices = np.where(players_x > 0.5)[0]

    draft_opt = np.array([indices[0] - len(player_arr)*1, indices[1] - len(player_arr)*2, 
                          indices[2] - len(player_arr)*3, indices[3] - len(player_arr)*4, 
                          indices[4] - len(player_arr)*5, indices[5] - len(player_arr)*6, 
                          indices[6] - len(player_arr)*7, indices[7] - len(player_arr)*8]
    )

    for i in range(len(draft_opt)):
        print(draft_opt[i].astype(str), ": ", player_arr[draft_opt[i]].astype(str))

In [127]:
# 5TH OVERALL PICK

m = Model()

draft_order = draft_11

# player dummy variable
x = m.addVars(len(player_arr), vtype=GRB.BINARY)

# draft round variable
y = m.addVars(len(draft_order), len(player_arr), vtype=GRB.BINARY)


# constraint: can't pick the same player twice
#for k in range(len(draft_order)):
 #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
for i in range(len(player_arr)):
    m.addConstr(sum(y[k,i] for k in range(len(draft_order))) <= 1)
    
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] for i in range(len(player_arr))) == 1) # constraint -- each round draft one player
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] * player_arr[i,5] for i in range(len(player_arr))) >= draft_order[k]) #ADP constraint
    #m.addConstr(y[k,i] == x[i])

# starting lineup has 8 players -- constraint
#m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

# constraint: maximum of 7 flex eligible players
#m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

#constraint: total of 8 players
m.addConstr(sum(y[k,i] for k in range(len(draft_order)) for i in range(len(player_arr))) == 8)



# QB constraint: 1 QB in lineup
m.addConstr(sum(y[k,i] * positions[i,0] for k in range(len(draft_order)) for i in range(len(player_arr))) == 1)
# # RB constraint: minimum of 2 RBs in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)
# # TE constraint: minimum of 1 TE in lineup, maximum of 3
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 1)
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 3)
# # WR constraint: minimum of 2 WR in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)

    
m.setObjective(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))), GRB.MAXIMIZE)


m.update()
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 374 rows, 3150 columns and 16424 nonzeros
Model fingerprint: 0x0bbf6206
Variable types: 0 continuous, 3150 integer (3150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [3e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective 713.9220000
Presolve removed 204 rows and 2123 columns
Presolve time: 0.12s
Presolved: 170 rows, 1027 columns, 4082 nonzeros
Found heuristic solution: objective 1739.7420000
Variable types: 0 continuous, 1027 integer (1027 binary)

Root relaxation: objective 1.756876e+03, 231 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1756.8760000 1756.87600  0.00%     -

In [1286]:
draft_order

array([ 5, 16, 25, 36, 45, 56, 65, 76])

In [1284]:
initial_pts = m.objval
initial_team = test_player_arr[draft_opt]

print("Initial Pts: ", initial_pts)
print("Initial Team: ", initial_team)

Initial Pts:  1874.818
Initial Team:  [['WR' 367.0 21.6 296.46999999999997 1.0 5 'Cooper Kupp LAR' 331.735]
 ['WR' 300.5 18.8 241.64999999999998 1.0 16 'Deebo Samuel SF' 271.075]
 ['QB' 417.7 24.6 387.538 0.0 41 'Josh Allen BUF' 402.619]
 ['WR' 194.6 11.4 199.52999999999997 1.0 34 'Michael Pittman Jr. IND'
  197.065]
 ['RB' 0.0 0.0 191.51 1.0 47 'J.K. Dobbins BAL' 95.755]
 ['WR' 186.8 11.7 194.25999999999996 1.0 51 'Brandin Cooks HOU'
  190.52999999999997]
 ['TE' 169.8 10.0 167.78 1.0 71 'Dalton Schultz DAL' 168.79000000000002]
 ['RB' 208.6 13.0 168.04000000000002 1.0 65 'Cordarrelle Patterson ATL'
  188.32]]


In [1282]:
end_pts = m.objval
end_team = player_arr[draft_opt]

print("End Pts: ", end_pts)
print("End Team: ", end_team)

End Pts:  1874.818
End Team:  [['WR' 367.0 21.6 296.46999999999997 1.0 3.5 'Cooper Kupp LAR' 331.735]
 ['WR' 300.5 18.8 241.64999999999998 1.0 18.8 'Deebo Samuel SF' 271.075]
 ['QB' 417.7 24.6 387.538 0.0 22.8 'Josh Allen BUF' 402.619]
 ['WR' 194.6 11.4 199.52999999999997 1.0 36.8 'Michael Pittman Jr. IND'
  197.065]
 ['RB' 0.0 0.0 191.51 1.0 54.8 'J.K. Dobbins BAL' 95.755]
 ['WR' 186.8 11.7 194.25999999999996 1.0 62.3 'Brandin Cooks HOU'
  190.52999999999997]
 ['TE' 169.8 10.0 167.78 1.0 65.0 'Dalton Schultz DAL' 168.79000000000002]
 ['RB' 208.6 13.0 168.04000000000002 1.0 87.0 'Cordarrelle Patterson ATL'
  188.32]]


In [1167]:
pts_rd2 = m.objval
team_rd2 = player_arr[draft_opt]

print("Initial Pts: ", pts_rd2)
print("Initial Team: ", team_rd2)

Initial Pts:  1838.594
Initial Team:  [['WR' 367.0 21.6 296.46999999999997 1.0 3.5 'Cooper Kupp LAR' 331.735]
 ['WR' 300.5 18.8 241.64999999999998 1.0 18.8 'Deebo Samuel SF' 271.075]
 ['RB' 239.2 15.9 221.36000000000004 1.0 31.3 'James Conner ARI'
  230.28000000000003]
 ['WR' 194.6 11.4 199.52999999999997 1.0 36.8 'Michael Pittman Jr. IND'
  197.065]
 ['RB' 0.0 0.0 191.51 1.0 54.8 'J.K. Dobbins BAL' 95.755]
 ['WR' 186.8 11.7 194.25999999999996 1.0 62.3 'Brandin Cooks HOU'
  190.52999999999997]
 ['TE' 169.8 10.0 167.78 1.0 65.0 'Dalton Schultz DAL' 168.79000000000002]
 ['QB' 386.7 22.7 326.034 0.0 81.5 'Tom Brady TB' 356.36699999999996]]


In [ ]:
pts_rd3 = m.objval
team_rd3 = player_arr[draft_opt]

print("Initial Pts: ", pts_rd3)
print("Initial Team: ", team_rd3)

In [823]:
# 6TH OVERALL PICK

m = Model()

draft_order = draft_6

# player dummy variable
x = m.addVars(len(player_arr), vtype=GRB.BINARY)

# draft round variable
y = m.addVars(len(draft_order), len(player_arr), vtype=GRB.BINARY)


# constraint: can't pick the same player twice
#for k in range(len(draft_order)):
 #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
for i in range(len(player_arr)):
    m.addConstr(sum(y[k,i] for k in range(len(draft_order))) <= 1)
    
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] for i in range(len(player_arr))) == 1) # constraint -- each round draft one player
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] * player_arr[i,5] for i in range(len(player_arr))) >= draft_order[k]) #ADP constraint
    #m.addConstr(y[k,i] == x[i])

# starting lineup has 8 players -- constraint
#m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

# constraint: maximum of 7 flex eligible players
#m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

#constraint: total of 8 players
m.addConstr(sum(y[k,i] for k in range(len(draft_order)) for i in range(len(player_arr))) == 8)



# QB constraint: 1 QB in lineup
m.addConstr(sum(y[k,i] * positions[i,0] for k in range(len(draft_order)) for i in range(len(player_arr))) == 1)
# # RB constraint: minimum of 2 RBs in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)
# # TE constraint: minimum of 1 TE in lineup, maximum of 3
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 1)
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 3)
# # WR constraint: minimum of 2 WR in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)

    
m.setObjective(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))), GRB.MAXIMIZE)


m.update()
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 703 rows, 6111 columns and 23928 nonzeros
Model fingerprint: 0xccd0256a
Variable types: 0 continuous, 6111 integer (6111 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e-01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective 596.8180000
Presolve removed 553 rows and 5482 columns
Presolve time: 0.06s
Presolved: 150 rows, 629 columns, 2486 nonzeros
Found heuristic solution: objective 1805.8700000
Variable types: 0 continuous, 629 integer (629 binary)

Root relaxation: objective 1.812490e+03, 238 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1812.4900000 1812.49000  0.00%     -   

In [824]:
players_x = np.asarray(m.x)
indices = np.where(players_x > 0.5)[0]

draft_opt = np.array([indices[0] - len(player_arr)*1, indices[1] - len(player_arr)*2, 
                      indices[2] - len(player_arr)*3, indices[3] - len(player_arr)*4, 
                      indices[4] - len(player_arr)*5, indices[5] - len(player_arr)*6, 
                      indices[6] - len(player_arr)*7, indices[7] - len(player_arr)*8]
)

for i in range(len(draft_opt)):
    print(player_arr[draft_opt[i]])

['WR' 276.4 16.3 260.28000000000003 1.0 6.5 'Justin Jefferson MIN']
['RB' 211.2 16.2 245.95 1.0 19.5 'Alvin Kamara NO']
['RB' 239.2 15.9 221.36000000000004 1.0 31.3 'James Conner ARI']
['QB' 374.2 22.0 361.36 0.0 35.5 'Patrick Mahomes II KC']
['WR' 208.6 13.0 193.72999999999996 1.0 48.3 'Mike Williams LAC']
['WR' 186.8 11.7 194.25999999999996 1.0 62.3 'Brandin Cooks HOU']
['RB' 104.2 8.7 181.37 1.0 70.8 'Miles Sanders PHI']
['TE' 137.0 9.1 154.18 1.0 78.8 'Dallas Goedert PHI']


In [825]:
# 7TH OVERALL PICK

m = Model()

draft_order = draft_7

# player dummy variable
x = m.addVars(len(player_arr), vtype=GRB.BINARY)

# draft round variable
y = m.addVars(len(draft_order), len(player_arr), vtype=GRB.BINARY)


# constraint: can't pick the same player twice
#for k in range(len(draft_order)):
 #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
for i in range(len(player_arr)):
    m.addConstr(sum(y[k,i] for k in range(len(draft_order))) <= 1)
    
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] for i in range(len(player_arr))) == 1) # constraint -- each round draft one player
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] * player_arr[i,5] for i in range(len(player_arr))) >= draft_order[k]) #ADP constraint
    #m.addConstr(y[k,i] == x[i])

# starting lineup has 8 players -- constraint
#m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

# constraint: maximum of 7 flex eligible players
#m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

#constraint: total of 8 players
m.addConstr(sum(y[k,i] for k in range(len(draft_order)) for i in range(len(player_arr))) == 8)



# QB constraint: 1 QB in lineup
m.addConstr(sum(y[k,i] * positions[i,0] for k in range(len(draft_order)) for i in range(len(player_arr))) == 1)
# # RB constraint: minimum of 2 RBs in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)
# # TE constraint: minimum of 1 TE in lineup, maximum of 3
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 1)
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 3)
# # WR constraint: minimum of 2 WR in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)

    
m.setObjective(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))), GRB.MAXIMIZE)


m.update()
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 703 rows, 6111 columns and 23928 nonzeros
Model fingerprint: 0xf18bfbcf
Variable types: 0 continuous, 6111 integer (6111 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e-01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective 596.8180000
Presolve removed 448 rows and 5176 columns
Presolve time: 0.06s
Presolved: 255 rows, 935 columns, 3714 nonzeros
Found heuristic solution: objective 1801.1800000
Variable types: 0 continuous, 935 integer (935 binary)

Root relaxation: objective 1.817800e+03, 365 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1817.8000000 1817.80000  0.00%     -   

In [826]:
players_x = np.asarray(m.x)
indices = np.where(players_x > 0.5)[0]

draft_opt = np.array([indices[0] - len(player_arr)*1, indices[1] - len(player_arr)*2, 
                      indices[2] - len(player_arr)*3, indices[3] - len(player_arr)*4, 
                      indices[4] - len(player_arr)*5, indices[5] - len(player_arr)*6, 
                      indices[6] - len(player_arr)*7, indices[7] - len(player_arr)*8]
)

for i in range(len(draft_opt)):
    print(player_arr[draft_opt[i]])

['WR' 264.1 15.5 255.59 1.0 9.0 "Ja'Marr Chase CIN"]
['TE' 216.8 13.6 236.62000000000003 1.0 14.0 'Travis Kelce KC']
['RB' 239.2 15.9 221.36000000000004 1.0 31.3 'James Conner ARI']
['QB' 374.2 22.0 361.36 0.0 35.5 'Patrick Mahomes II KC']
['WR' 208.6 13.0 193.72999999999996 1.0 48.3 'Mike Williams LAC']
['WR' 186.8 11.7 194.25999999999996 1.0 62.3 'Brandin Cooks HOU']
['RB' 104.2 8.7 181.37 1.0 70.8 'Miles Sanders PHI']
['WR' 179.2 10.5 173.51 1.0 75.8 'Darnell Mooney CHI']


In [827]:
# 8TH OVERALL PICK

m = Model()

draft_order = draft_8

# player dummy variable
x = m.addVars(len(player_arr), vtype=GRB.BINARY)

# draft round variable
y = m.addVars(len(draft_order), len(player_arr), vtype=GRB.BINARY)


# constraint: can't pick the same player twice
#for k in range(len(draft_order)):
 #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
for i in range(len(player_arr)):
    m.addConstr(sum(y[k,i] for k in range(len(draft_order))) <= 1)
    
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] for i in range(len(player_arr))) == 1) # constraint -- each round draft one player
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] * player_arr[i,5] for i in range(len(player_arr))) >= draft_order[k]) #ADP constraint
    #m.addConstr(y[k,i] == x[i])

# starting lineup has 8 players -- constraint
#m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

# constraint: maximum of 7 flex eligible players
#m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

#constraint: total of 8 players
m.addConstr(sum(y[k,i] for k in range(len(draft_order)) for i in range(len(player_arr))) == 8)



# QB constraint: 1 QB in lineup
m.addConstr(sum(y[k,i] * positions[i,0] for k in range(len(draft_order)) for i in range(len(player_arr))) == 1)
# # RB constraint: minimum of 2 RBs in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)
# # TE constraint: minimum of 1 TE in lineup, maximum of 3
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 1)
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 3)
# # WR constraint: minimum of 2 WR in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)

    
m.setObjective(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))), GRB.MAXIMIZE)


m.update()
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 703 rows, 6111 columns and 23928 nonzeros
Model fingerprint: 0xae011654
Variable types: 0 continuous, 6111 integer (6111 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e-01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective 596.8180000
Presolve removed 554 rows and 5388 columns
Presolve time: 0.06s
Presolved: 149 rows, 723 columns, 2864 nonzeros
Found heuristic solution: objective 1801.4340000
Variable types: 0 continuous, 723 integer (723 binary)

Root relaxation: objective 1.818054e+03, 239 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1818.0540000 1818.05400  0.00%     -   

In [828]:
players_x = np.asarray(m.x)
indices = np.where(players_x > 0.5)[0]

draft_opt = np.array([indices[0] - len(player_arr)*1, indices[1] - len(player_arr)*2, 
                      indices[2] - len(player_arr)*3, indices[3] - len(player_arr)*4, 
                      indices[4] - len(player_arr)*5, indices[5] - len(player_arr)*6, 
                      indices[6] - len(player_arr)*7, indices[7] - len(player_arr)*8]
)

for i in range(len(draft_opt)):
    print(player_arr[draft_opt[i]])

['WR' 264.1 15.5 255.59 1.0 9.0 "Ja'Marr Chase CIN"]
['TE' 216.8 13.6 236.62000000000003 1.0 14.0 'Travis Kelce KC']
['RB' 239.2 15.9 221.36000000000004 1.0 31.3 'James Conner ARI']
['RB' 175.0 12.5 209.08999999999997 1.0 33.5 'David Montgomery CHI']
['WR' 208.6 13.0 193.72999999999996 1.0 48.3 'Mike Williams LAC']
['WR' 186.8 11.7 194.25999999999996 1.0 62.3 'Brandin Cooks HOU']
['RB' 104.2 8.7 181.37 1.0 70.8 'Miles Sanders PHI']
['QB' 386.7 22.7 326.034 0.0 81.5 'Tom Brady TB']


In [829]:
# 9TH OVERALL PICK

m = Model()

draft_order = draft_9

# player dummy variable
x = m.addVars(len(player_arr), vtype=GRB.BINARY)

# draft round variable
y = m.addVars(len(draft_order), len(player_arr), vtype=GRB.BINARY)


# constraint: can't pick the same player twice
#for k in range(len(draft_order)):
 #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
for i in range(len(player_arr)):
    m.addConstr(sum(y[k,i] for k in range(len(draft_order))) <= 1)
    
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] for i in range(len(player_arr))) == 1) # constraint -- each round draft one player
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] * player_arr[i,5] for i in range(len(player_arr))) >= draft_order[k]) #ADP constraint
    #m.addConstr(y[k,i] == x[i])

# starting lineup has 8 players -- constraint
#m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

# constraint: maximum of 7 flex eligible players
#m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

#constraint: total of 8 players
m.addConstr(sum(y[k,i] for k in range(len(draft_order)) for i in range(len(player_arr))) == 8)



# QB constraint: 1 QB in lineup
m.addConstr(sum(y[k,i] * positions[i,0] for k in range(len(draft_order)) for i in range(len(player_arr))) == 1)
# # RB constraint: minimum of 2 RBs in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)
# # TE constraint: minimum of 1 TE in lineup, maximum of 3
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 1)
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 3)
# # WR constraint: minimum of 2 WR in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)

    
m.setObjective(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))), GRB.MAXIMIZE)


m.update()
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 703 rows, 6111 columns and 23928 nonzeros
Model fingerprint: 0x0add63e6
Variable types: 0 continuous, 6111 integer (6111 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e-01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective 596.8180000
Presolve removed 442 rows and 4957 columns
Presolve time: 0.06s
Presolved: 261 rows, 1154 columns, 4592 nonzeros
Found heuristic solution: objective 1808.6600000
Variable types: 0 continuous, 1154 integer (1154 binary)

Root relaxation: objective 1.823060e+03, 363 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1823.0600000 1823.06000  0.00%     -

In [830]:
players_x = np.asarray(m.x)
indices = np.where(players_x > 0.5)[0]

draft_opt = np.array([indices[0] - len(player_arr)*1, indices[1] - len(player_arr)*2, 
                      indices[2] - len(player_arr)*3, indices[3] - len(player_arr)*4, 
                      indices[4] - len(player_arr)*5, indices[5] - len(player_arr)*6, 
                      indices[6] - len(player_arr)*7, indices[7] - len(player_arr)*8]
)

for i in range(len(draft_opt)):
    print(player_arr[draft_opt[i]])

['WR' 264.1 15.5 255.59 1.0 9.0 "Ja'Marr Chase CIN"]
['TE' 216.8 13.6 236.62000000000003 1.0 14.0 'Travis Kelce KC']
['RB' 239.2 15.9 221.36000000000004 1.0 31.3 'James Conner ARI']
['QB' 374.2 22.0 361.36 0.0 35.5 'Patrick Mahomes II KC']
['WR' 186.8 11.7 194.25999999999996 1.0 62.3 'Brandin Cooks HOU']
['RB' 0.0 0.0 191.51 1.0 54.8 'J.K. Dobbins BAL']
['RB' 104.2 8.7 181.37 1.0 70.8 'Miles Sanders PHI']
['WR' 182.3 11.4 180.99 1.0 72.3 'Amon-Ra St. Brown DET']


In [831]:
# 10TH OVERALL PICK

m = Model()

draft_order = draft_10

# player dummy variable
x = m.addVars(len(player_arr), vtype=GRB.BINARY)

# draft round variable
y = m.addVars(len(draft_order), len(player_arr), vtype=GRB.BINARY)


# constraint: can't pick the same player twice
#for k in range(len(draft_order)):
 #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
for i in range(len(player_arr)):
    m.addConstr(sum(y[k,i] for k in range(len(draft_order))) <= 1)
    
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] for i in range(len(player_arr))) == 1) # constraint -- each round draft one player
    
for k in range(len(draft_order)):
    m.addConstr(sum(y[k,i] * player_arr[i,5] for i in range(len(player_arr))) >= draft_order[k]) #ADP constraint
    #m.addConstr(y[k,i] == x[i])

# starting lineup has 8 players -- constraint
#m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

# constraint: maximum of 7 flex eligible players
#m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

#constraint: total of 8 players
m.addConstr(sum(y[k,i] for k in range(len(draft_order)) for i in range(len(player_arr))) == 8)



# QB constraint: 1 QB in lineup
m.addConstr(sum(y[k,i] * positions[i,0] for k in range(len(draft_order)) for i in range(len(player_arr))) == 1)
# # RB constraint: minimum of 2 RBs in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)
# # TE constraint: minimum of 1 TE in lineup, maximum of 3
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 1)
m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 3)
# # WR constraint: minimum of 2 WR in lineup, maximum of 4
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) >= 2)
m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 4)

    
m.setObjective(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))), GRB.MAXIMIZE)


m.update()
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 703 rows, 6111 columns and 23928 nonzeros
Model fingerprint: 0x7b202402
Variable types: 0 continuous, 6111 integer (6111 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e-01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Found heuristic solution: objective 596.8180000
Presolve removed 410 rows and 4093 columns
Presolve time: 0.05s
Presolved: 293 rows, 2018 columns, 8056 nonzeros
Found heuristic solution: objective 1787.4940000
Variable types: 0 continuous, 2018 integer (2018 binary)

Root relaxation: objective 1.813420e+03, 382 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1813.4200000 1813.42000  0.00%     -

In [832]:
players_x = np.asarray(m.x)
indices = np.where(players_x > 0.5)[0]

draft_opt = np.array([indices[0] - len(player_arr)*1, indices[1] - len(player_arr)*2, 
                      indices[2] - len(player_arr)*3, indices[3] - len(player_arr)*4, 
                      indices[4] - len(player_arr)*5, indices[5] - len(player_arr)*6, 
                      indices[6] - len(player_arr)*7, indices[7] - len(player_arr)*8]
)

for i in range(len(draft_opt)):
    print(player_arr[draft_opt[i]])

['TE' 216.8 13.6 236.62000000000003 1.0 14.0 'Travis Kelce KC']
['RB' 211.2 16.2 245.95 1.0 19.5 'Alvin Kamara NO']
['RB' 239.2 15.9 221.36000000000004 1.0 31.3 'James Conner ARI']
['QB' 374.2 22.0 361.36 0.0 35.5 'Patrick Mahomes II KC']
['WR' 186.8 11.7 194.25999999999996 1.0 62.3 'Brandin Cooks HOU']
['RB' 0.0 0.0 191.51 1.0 54.8 'J.K. Dobbins BAL']
['RB' 104.2 8.7 181.37 1.0 70.8 'Miles Sanders PHI']
['WR' 182.3 11.4 180.99 1.0 72.3 'Amon-Ra St. Brown DET']


In [1004]:
# 5TH OVERALL PICK

m = Model()

draft_order = np.array([5,20,29,44,53,68,77,92])
draft_order_bn = np.array([101,116,125,140,149,164,173])

rounds = 15
starters = 8

# player dummy variable
x = m.addVars(len(test_player_arr), vtype=GRB.BINARY)

# draft round variable
y = m.addVars(len(draft_order_bn), len(test_player_arr), vtype=GRB.BINARY)


# constraint: can't pick the same player twice
#for k in range(len(draft_order)):
 #   m.addGenConstrAbs(y[k,i], sum(y[j,i] for i in range(len(player_arr)) for j in range(len(draft_order)) if j != k) >= 0.5)
for i in range(len(test_player_arr)):
    m.addConstr(sum(y[k,i] for k in range(len(draft_order_bn))) <= 1)
    
    
for k in range(len(draft_order_bn)):
    m.addConstr(sum(y[k,i] for i in range(len(test_player_arr))) == 1) # constraint -- each round draft one player
    
for k in range(len(draft_order_bn)):
    m.addConstr(sum(y[k,i] * test_player_arr[i,5] for i in range(len(test_player_arr))) >= draft_order_bn[k]) #ADP constraint
    #m.addConstr(y[k,i] == x[i])

# starting lineup has 8 players -- constraint
#m.addConstr(sum(x[i] for i in range(len(player_arr))) == 8)

# constraint: maximum of 7 flex eligible players
#m.addConstr(sum(y[k,i] * player_arr[i,3] for k in range(len(draft_order)) for i in range(len(player_arr))) <= 7)

#constraint: total of 8 players
m.addConstr(sum(y[k,i] for k in range(len(draft_order_bn)) for i in range(len(test_player_arr))) == 7)



# QB constraint: 1 QB in lineup
m.addConstr(sum(y[k,i] * positions[i,0] for k in range(len(draft_order_bn)) for i in range(len(test_player_arr))) == 1)
# # RB constraint: minimum of 2 RBs in lineup, maximum of 4
# m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(test_player_arr))) >= 2)
# m.addConstr(sum(y[k,i] * positions[i,1] for k in range(len(draft_order)) for i in range(len(test_player_arr))) <= 4)
# # TE constraint: minimum of 1 TE in lineup, maximum of 3
# m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(test_player_arr))) >= 1)
# m.addConstr(sum(y[k,i] * positions[i,2] for k in range(len(draft_order)) for i in range(len(test_player_arr))) <= 3)
# # WR constraint: minimum of 2 WR in lineup, maximum of 4
# m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(test_player_arr))) >= 2)
# m.addConstr(sum(y[k,i] * positions[i,3] for k in range(len(draft_order)) for i in range(len(test_player_arr))) <= 4)

    
m.setObjective(sum(y[k,i] * test_player_arr[i,3] for k in range(len(draft_order_bn)) for i in range(len(player_arr))), GRB.MAXIMIZE)


m.update()
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 349 rows, 2664 columns and 9639 nonzeros
Model fingerprint: 0xa73a15a0
Variable types: 0 continuous, 2664 integer (2664 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [2e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 642.8840000
Presolve removed 318 rows and 2573 columns
Presolve time: 0.02s
Presolved: 31 rows, 91 columns, 177 nonzeros
Found heuristic solution: objective 1138.2280000
Variable types: 0 continuous, 91 integer (91 binary)

Root relaxation: cutoff, 47 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1138.22800 1138.22800  0.00%     -    0s

Explored 0 nodes (

In [1007]:
players_x = np.asarray(m.x)
indices = np.where(players_x > 0.5)[0]

draft_opt = np.array([indices[0] - len(player_arr)*1, indices[1] - len(player_arr)*2, 
                      indices[2] - len(player_arr)*3, indices[3] - len(player_arr)*4, 
                      indices[4] - len(player_arr)*5, indices[5] - len(player_arr)*6, 
                      indices[6] - len(player_arr)*7]
)

for i in range(len(draft_opt)):
    print(player_arr[draft_opt[i]])

['QB' 67.0 11.2 302.52799999999996 0.0 104.3 'Trey Lance SF']
['WR' 137.1 9.1 150.17 1.0 116.5 'Chase Claypool PIT']
['WR' 70.6 5.0 133.06 1.0 151.0 'Kenny Golladay NYG']
['TE' 139.3 8.7 142.81 1.0 169.8 'Hunter Henry NE']
['WR' 144.8 9.1 140.26 1.0 153.7 'Jakobi Meyers NE']
['WR' 111.4 6.6 136.89 1.0 166.0 'Mecole Hardman KC']
['TE' 125.0 7.8 132.51 1.0 179.3 'Noah Fant SEA']
